# 离子阱抗噪声Mølmer-Sørensen双比特门

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*


## 内容概要
本教程中，我们将介绍如何使用量脉在离子阱量子计算中实现抗噪声的 Mølmer-Sørensen 双比特门操作。本教程的大纲如下：


- 离子阱抗噪声 Mølmer-Sørensen 双比特门概念介绍
- 离子阱 Mølmer-Sørensen 双比特门保真度
- 离子阱 Mølmer-Sørensen 双比特门噪声
- Mølmer-Sørensen 门抗噪声方法
- 离子阱抗噪声 Mølmer-Sørensen 双比特门脉冲生成
- 总结
- 参考文献

## 离子阱 Robust Mølmer-Sørensen 双比特门概念介绍

在离子阱平台中，双比特门是通过 Mølmer-Sørensen（MS） 相互作用实现的。该作用通过等效拉曼光作用在单个离子上而将其与离子链的声子模耦合，在一个激光作用周期内，若两个离子分别与声子模 $\omega_k$ 耦合再解耦，等效看来，相当于两个离子通过交换声子而间接耦合起来。离子链中交换声子而形成的两个离子之间的等效相互作用我们称为 MS 相互作用，由此构成的双比特门被称作 MS 门。可以看出，我们希望在MS门作用后保留的是离子 $i$ 与离子 $j$ 的耦合，用 $\phi_{ij}$ 表示；希望消除的是离子-声子耦合，用 $\alpha_{i,k} ,  \alpha_{j,k}$ 表示。虽然离子阱的单双量子门操作是目前不同量子计算硬件平台里精度最高的，但要做到实验上的广泛应用，则需要对现阶段实验仪器的 dephasing 、 timing 两种噪声有很强抗干扰能力。 其中， dephasing 噪声一般指激光线宽过宽、频率抖动较严重等因素影响下，拉曼光驱动的离子阱 MS 门不能让所有的声子-离子耦合强度 $\alpha_{i,k}$ 在门作用时间结束后回到解耦状态，从而导致的门保真度降低。 而 timing 噪声则指的是仪器真实的脉冲作用时间会有抖动，从而造成的门保真度的降低。

## 离子阱 Mølmer-Sørensen 双比特门保真度

在离子阱平台的硬件层面，每个参与相互作用的离子都需要两束拉曼光。对第 $j$ 个离子，红光与蓝光的相位分别是 $\phi^r_j$ 与 $\phi^b_j$ 。设 $\phi^S_j=(\phi^r_j+\phi^b_j)/2$ ， $\phi^M_j=(\phi^r_j-\phi^b_j)/2$， MS 相互作用可以表示为 Bloch Sphere 上绕 $\phi^S_j$ 轴旋转的哈密顿量：

$$
H_{\mathbf{MS}}=\sum_j\sum_{k=0}^{N-1}\hbar\eta_{k,j}\Omega_j\cos(\mu t-\phi_j^M)(\hat{a}_ke^{-i\omega_k t}+\hat{a}_k^\dagger e^{i\omega_k t})\hat{\sigma}_\phi^{(j)}.
$$

其等效Pauli矩阵为 $\hat{\sigma}_\phi^{(j)}=\cos(\phi^S_j)\sigma_x^{(j)}+\sin(\phi^S_j)\sigma_y^{(j)}$，其中， $\eta_{k,j}$ 是离子 $j$ 与声子模 $k$ 的耦合参数，称之为 Lamb-Dicke 系数； $\Omega_j$ 是离子 $j$ 在拉曼光作用下的等效拉比振荡频率，与拉曼光振幅成正相关； $\mu$ 是拉曼光失谐，大小为拉曼频率与 $|0\rangle$,$|1\rangle$ 态的能量差； $\hat{a}_k$ 是声子模 $k$ 的湮灭算符。在这样的作用下，使用 Magnus 展开写出离子-声子量子态随时间的演化矩阵：

$$
U(\tau)=\exp\left[-\frac{i}{\hbar}\int_0^\tau dtH_{\mathbf{MS}}(t)-\frac{1}{2\hbar^2}\int_0^\tau dt_2\int_0^{t_2}dt_1[H_{\mathbf{MS}}(t_2),H_{\mathbf{MS}}(t_1)]+...\right].
$$

带入计算，可得到等效的时间演化算符表达式：

$$
U(\tau)=\sum_{j}\sum_{k=0}^{N-1}(\alpha_{j,k}(\tau)\hat{a}^\dagger+\alpha_{j,k}^*(\tau)\hat{a})\hat{\sigma}_\phi^{(j)}+i\sum_{m\neq n}\chi_{m,n}(\tau)\hat{\sigma}_\phi^{(m)}\hat{\sigma}_\phi^{(n)},
$$

其中 $\alpha_{j,k}$ 代表离子 $j$ 与声子 $k$ 的耦合强度， $\chi_{m,n}$ 是离子 $m$ 与离子 $n$ 的耦合强度，表达式为：

$$
\alpha_{j,k}(\tau)=-\eta_{k,j}\int_0^\tau\Omega_j(t)\sin(\mu t-\phi_j^M)e^{i\omega_kt}dt,
$$
$$
\chi_{m,n}(\tau)=\sum_{k=0}^{N-1}\eta_{k,m}\eta_{k,n}\int_0^\tau dt_2\int_0^{t_2}dt_1\left[\Omega_m(t_2)\Omega_n(t_1)\sin\omega_k(t_2-t_1)\cos(\mu t_2-\phi_m^M)\cos(\mu t_1-\phi_n^M)\right].
$$

我们的操作目的是实现离子间的双量子比特门，如果操作结束后保留任何一个离子-声子耦合强度 $\alpha_{j,k}$ ，则对离子-声子完全耦合的系统中所有声子模求迹后，我们关心的离子量子比特状态会处于混态，量子比特间的信息会有一部分和声子模产生纠缠，残留在声子上的信息无法提取，导致离子阱量子比特门保真度的丢失。在此基础上，对 MS 门保真度影响最大的就是声子-离子耦合 $\alpha_{i,k}$ 不能在一个量子门操作结束后归零的问题，一切改善离子阱双量子比特门保真度的尝试都围绕此目的展开。对 MS 门，若我们想达到两个离子间耦合强度为 $\phi$ ，其一般保真度的表达式为：

$$
F=\frac{1}{8}[2+2\cos(2(\chi_{m,n}-\phi))(\Gamma_m+\Gamma_n)+(\Gamma_-+\Gamma_+)],
$$

公式内的每一项表示为：

$$
\Gamma_{m(n)}=e^{-\sum_{k=0}^{N-1}(|\alpha_{m(n)),k}(\tau)|^2\beta_k/2},
$$

$$
\Gamma_{\pm}=e^{-\sum_{k=0}^{N-1}(|\alpha_{m,k}(\tau)\pm\alpha_{n,k}(\tau)|^2\beta_k/2)},
$$

$$
\beta_k=\coth\left[\frac{1}{2}\ln(1+1/\bar{n}_k(T))\right].
$$

其中， $\bar{n}_k(T)$ 为声子模 $\omega_k$ 在离子链温度为 $T$ 时的平均声子数。


## 离子阱 Mølmer-Sørensen 双比特门噪声

我们使用如下例子来说明 dephasing 和 timing 两种噪声是如何对离子-声子耦合强度 $\alpha$ 产生影响的。我们考虑作用在离子 $j$ 上的振幅为 $\Omega$ 、相位为 $\phi$ 、 Raman 失谐为 $\mu$ 、作用时间为 $\tau$ 的恒定脉冲。对于 MS 相互作用，该脉冲会将离子与声子模耦合起来。对声子模 $\omega_k$ ，积分得到脉冲作用后的离子-声子耦合强度为：

$$
\alpha_{j,k}(\tau)=\frac{\eta_{k,j}\Omega_j}{\mu^2-\omega_k^2}[e^{i\omega_k\tau}(\mu\cos(\mu\tau-\phi_j^M)-i\omega_k\sin(\mu\tau-\phi_j^M))-(\mu\cos\phi_j^M+i\omega_k\sin\phi_j^M)].
$$

注意到当 $|\mu-\omega_k|\ll\mu+\omega_k$ ，结果在旋波近似下可写成：

$$
\alpha(\tau)\approx\frac{\eta_{k,j}\Omega_j}{2}(\frac{e^{i[(\omega_k-\mu)\tau+\phi_j^M]}-1}{\mu-\omega_k}).
$$

可以看出，当方程满足 $(\omega_k-\mu)\tau+\phi_j^M=2n\pi$ 的时候 $\alpha(\tau)=0$ ，表现为 $\alpha$ 在 $\tau$ 时间内绕回了原点，我们称这个恒定脉冲满足声子解耦条件。在此基础上，若系统受到外部扰动，使得 $\mu$ 的大小漂移（ dephasing 噪声）、 $\tau$ 的长度伸缩（ timing 噪声），则会使得 $\alpha$ 在 $\tau$ 时间内没有绕回原点，如图所示：

![noise_alpha](figures/tutorial_noise.png)

左图中，我们使用了符合声子解耦条件的参数，其中 $\mu$=1.6MHz, $\tau$=63 $\mu s$。而右图中 dephasing noise 的 $\mu$=1.48MHz ，timing noise 的 $\tau$=53.6 $\mu s$ 相较于真实值分别漂移了 16KHz 和 15%，引起声子-离子耦合残留上升，从而引起量子门保真度的降低。

## Mølmer-Sørensen 门抗噪声方法

上面我们看到了一个平稳脉冲在应对拉曼光频率、脉冲作用时间的整体性漂移噪声时，离子-声子耦合强度 $\alpha$ 不能在脉冲结束后回到零的例子。如果我们对脉冲做一些处理就能一定程度解决这个问题。比如，我们可以将脉冲设计成前后对称的两个部分，利用对称性去消除一部分整体漂移对 $\alpha$ 的影响。

我们使用一个二离子系统作为例子。我们可以对作用在这两个离子上的脉冲做切片操作，将其分成可以自由调节振幅、相位的 15 片脉冲，并使前 7 片脉冲和后 7 片脉冲的振幅对称、相位反对称，如图：

![Sympulse](figures/pulse_show.png)

计算脉冲对离子的作用，并根据需求调节每片脉冲的振幅与相位，将调好的脉冲作用在 2 个目标离子上，可以得到 2 个离子和 2 个声子模的耦合强度 $\alpha_{j,k}$ 的轨迹图：

![alphaSym](figures/alpha_show.png)

可以看出，经过我们的对称化操作后，离子-声子耦合强度 $\alpha$ 的轨迹变得对称且迂回，这样的轨迹在经受 dephasing, timing 的噪声时能表现出良好的抗噪特性。本教程讲述的方法就是围绕这个核心思想进行实现的。

## 离子阱抗噪声 Mølmer-Sørensen 双比特门脉冲生成

我们此处演示一个在含有10个离子的一维离子链中，使用对称切片优化的方法来生成抗噪声双量子 MS 门脉冲的示例。为此，我们首先需要从量脉中导入相应模块：

In [ ]:
# Import Robust Mølmer-Sørensen pulse module
from Quanlse.TrappedIon.QIonSystem import QChain1D, QLaser
from Quanlse.TrappedIon.Optimizer.OptimizerIon import optimizeIonSymmetry
import numpy as np

首先，我们需要定义离子阱芯片的环境信息以及拉曼光的基本信息。对离子阱芯片，我们要设定的物理量包括一维离子阱中离子数目 `ionNumber` 、要作用双比特门的两个离子位置 `indexIon` 、阱中离子的种类（质量） `mass` 、一维离子阱的横向简谐束缚频率 $\omega_{XY}$ 和纵向简谐束缚势 $\omega_z$ 以及进行冷却后得到的离子链温度 `temperature` 。而对于拉曼光，我们需要设定波长 `wavelength` 、拉曼光失谐 `detuning` 、两束拉曼光夹角 `laserAngle` 、作用门时间 $\tau$ 、对称优化的切片数量 `segments` 和实验上能达到的最大拉比频率 `omegaMax` （本示例中的参数参考自实验文章\[1\]）：

In [ ]:
# set experiment parameters
# ion trap
ionNumber = 10
indexIon = [0, 1]           # ion that interact with the laser pulse
mass = 171                  # atom mass
omegaXY = 22.5e6            # unit: Hz
omegaZ = 3.8e6              # unit: Hz
temperature = 1e-6          # unit: K
# laser
waveLength = 369.75         # unit: nm
detuning = 3.804e6          # unit: Hz
laserAngle = np.pi / 2      # angle between two laser beams
tau = 2e-4                  # unit: s
segments = 15               # laser pulse segments
omegaMax = 62.8e6           # unit: Hz

随后使用上述参数，从量脉中构建一维离子阱芯片和与离子作用的拉曼光的物理对象 `ionCain` 和 `ionLaser`：

In [ ]:
# generate the entity of ion chip and laser
ionChain = QChain1D(ionMass=mass, ionNumber=ionNumber, trapZ=omegaZ, trapXY=omegaXY, temperature=temperature)
ionLaser = QLaser(waveLength=waveLength, laserAngle=laserAngle, segments=segments, detuning=detuning, maxRabi=omegaMax,
                  tg=tau)   

设定预期的量子门抗噪声范围 `noise` ，将一维离子阱 `ionChain` 、拉曼光 `ionLaser` 、作用离子 `indexIon` 、噪声范围 `noise` 输入对称切片优化函数进行优化，得到优化后的拉曼光脉冲序列 `laserFinal` ：

In [ ]:
# use the symmetry method to optimize the laser pulse sequence to be dephasing robust
dephasingNoise = 2e3
laserFinal = optimizeIonSymmetry(ionChip=ionChain, laser=ionLaser, indexIon=indexIon, noise=dephasingNoise)

最后，我们检验该脉冲的抗噪声表现与离子耦合强度的变化。我们使用函数 `noiseFeature` 来一次性生成离子间耦合的变化、两种噪声对保真度的影响和作用在两个离子上的脉冲波形。设置门时间范围噪声 `timingNoise` ，将一维离子阱、优化好的拉曼光脉冲、作用离子、两种噪声输入函数：

In [ ]:
# show noise features using plot function
from Quanlse.TrappedIon.QIonTrajectory import noiseFeature, allAlphaComb
timingNoise = 0.001
noiseFeature(ionChip=ionChain, laser=laserFinal, indexIon=indexIon, noise=dephasingNoise, timeNoise=timingNoise)

其中，左上角子图代表离子-离子耦合在优化脉冲作用下的演化趋势，左下角子图代表 MS 门随 dephasing 噪声的改变。中间四图分别表示被要操作的两个离子上，激光脉冲振幅和相位的切片形式。右上角子图代表离子-离子耦合强度随 dephasing 噪声的改变，右下角代表 MS 门随 timing 噪声的改变。可以看出，本方法优化出的脉冲序列具有拉比频率（对应拉曼光强度）对称、拉曼光相位反对称的性质，且拉比频率被限制在最大拉比频率之内，保证了实验上的可行性。在合理的此类脉冲作用下，离子-离子耦合强度 $\chi$ 在门时间 $\tau$ 内缓慢上升到期望值 $\pi/4$ 。而在 dephasing noise、timing noise 的作用下，该量子门依然保持很低失真度。

关于对保真度影响最大的离子-声子耦合强度 $\alpha$ ，我们也给出了函数 `allAlphaComb` 将其全部表现在一张图中：

In [ ]:
# show all alpha trajectory
allAlphaComb(ionChip=ionChain, laser=laserFinal, index=indexIon)

我们可以方便地得到各离子-声子耦合强度 $\alpha$ 的变化轨迹图, 可以看出，在脉冲序列作用结束后，各个离子-声子模耦合强度都回到了原点。其中，与拉曼光失谐 `detuning` 最接近的声子模的耦合项画出的蓝色轨迹是对保真度影响最大的，在对称脉冲的作用下，该轨迹在一个门时间后回到原点，这很好体现了我们优化方法的原理。

## 总结

通过使用量脉，我们可以十分方便地构建离子阱平台和控制脉冲的对象，并在自定义设置的脉冲条件下生成想要的抗噪声脉冲。我们开发的简便可视化模块可以轻松显示人们关心的各种参量如离子-离子耦合强度 $\chi$ 的变化和离子-声子耦合 $\alpha$ 的轨迹。用户可以通过点击链接 [tutorial-trapped-ion-robust-control.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-trapped-ion-robust-control.ipynb) 跳转到此 Jupyter Notebook 文档相应的 GitHub 页面并且运行这个程序。

本方法在许多参数条件下都能有效运行，我们鼓励用户尝试不同于本教程的合理参数值以进一步探索量脉离子阱模块的功能。

## 参考文献
\[1\] [K. Kim, et al. "Entanglement and Tunable Spin-Spin Couplings Between Trapped Ions Using Multiple Transverse Modes." *Physical Review Letters* 105, 109901 (2010)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.120502)
